In [1]:
import d4rl
import gym
import numpy as np
from scipy.stats import norm

env = gym.make('maze2d-umaze-v1')
dataset = env.get_dataset()


/home/users/adarsh/anaconda3/envs/diffuser/lib/python3.8/site-packages/glfw/__init__.py:916: GLFWError: (65544) b'X11: The DISPLAY environment variable is missing'
  warnings.warn(message, GLFWError)
pybullet build time: May 20 2022 19:44:17
load datafile: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:00<00:00, 31.38it/s]


In [2]:
t=[]
max_t = t
epsilon = 0.3

for j, (x,y) in enumerate(dataset['infos/qvel']):
    if len(t) == 0:
        t.append([j,x,y])
    else:
        flag = 1
        idx = -1
        for i, (n,a,b) in enumerate(t):
            if abs(x-a) <= epsilon and abs(y-b) <= epsilon:
                continue
            else:
                flag = 0
                idx = i 
                break
        if flag:
            t.append([j,x,y])
        else:
            if len(t) > len(max_t):
                max_t = t
            t = t[i+1:]

In [3]:
idxs = np.array(max_t).astype(int)[:,0]

In [4]:
{k:v[idxs] for k,v in dataset.items()}

{'actions': array([[ 0.47399944,  0.7377699 ],
        [ 0.9628953 , -0.43183213],
        [ 0.39009994,  0.46581778],
        [ 0.66205287, -0.43900973],
        [ 1.        , -0.05332825],
        [ 1.        , -0.5726057 ],
        [-0.03002625,  0.5529734 ],
        [ 0.65565026,  0.2258165 ],
        [-0.4390777 , -0.48349717]], dtype=float32),
 'infos/goal': array([[2.9839995, 3.061838 ],
        [2.9839995, 3.061838 ],
        [2.9839995, 3.061838 ],
        [2.9839995, 3.061838 ],
        [2.9839995, 3.061838 ],
        [2.9839995, 3.061838 ],
        [2.9839995, 3.061838 ],
        [2.9839995, 3.061838 ],
        [2.9839995, 3.061838 ]], dtype=float32),
 'infos/qpos': array([[2.1916683, 2.9929364],
        [2.242678 , 2.9959104],
        [2.2948523, 2.997849 ],
        [2.345662 , 3.0008922],
        [2.3971198, 3.0028827],
        [2.4493823, 3.0047414],
        [2.501645 , 3.0052319],
        [2.5514543, 3.0070384],
        [2.6027067, 3.0093782]], dtype=float32),
 'infos/qv

In [19]:
import numpy as np
import scipy.stats
import torch
from scipy.stats import norm

class BasePrior:
    def __init__(self, transition_dim, horizon):
        super().__init__()
        self.transition_dim = transition_dim
        self.horizon = horizon

    def __call__(self, cond, device):
        batch_size = len(cond[0])
        shape = (batch_size, self.horizon, self.transition_dim)
        x = torch.randn(shape, device=device, dtype=torch.float32)
        return x


class Maze2DGoalPrior(BasePrior):
    def __init__(self, transition_dim, horizon):
        super().__init__(transition_dim, horizon)
        nd = np.array([norm.pdf(x, (horizon + 1)/2, horizon/6) for x in np.linspace(1, horizon, horizon)]).reshape((horizon, 1))
        self.normal_distribution = (nd - nd.min()) / nd.max()

    def __call__(self, cond):
        batch_size = len(cond[0])
        shape = (batch_size, self.horizon, self.transition_dim)
        start, goal = cond[0].detach().cpu().numpy(), cond[self.horizon-1].detach().cpu().numpy()

        actions, _ = self.get_action(start, goal)
        qpos = self.get_qpos(start, goal)
        qvel = self.get_qvel(start, goal)
        trajectory = np.concatenate((actions, qpos, qvel), axis=2)
#         x1 = torch.tensor(trajectory, device=device, dtype=torch.float32)
#         assert x1.shape == shape
        return trajectory

    def get_action(self, start, goal):
        delta = goal - start
        slope_rads = np.arctan2(delta[:, 1], delta[:, 0])
        action = np.column_stack((np.cos(slope_rads), np.sin(slope_rads)))
        action_normed = action / np.abs(action).max(axis=1)[:, np.newaxis]
        actions = np.repeat(action_normed[:, np.newaxis, :], self.horizon, axis=1)
        return actions, action_normed

    def get_qpos(self, start, goal):
        delta = (goal - start)/(self.horizon-1)
        return np.array([[s[:2] + i*d[:2] for i in range(0,self.horizon)] for d, s in zip(delta,start)], dtype=np.float32)

    def get_qvel(self, start, goal):
        mag = 100 * (goal - start) / (self.horizon - 1)
        _, action_normed = self.get_action(start, goal)
        qvel = mag[:, :2] * action_normed
        qvels = np.repeat(qvel[:, np.newaxis, :], self.horizon, axis=1)
        qvels = np.array([qvels[i]*self.normal_distribution for i in range(0,len(qvels))])
        return qvels



In [34]:
horizon = 15
batch_size = 7

start = np.random.rand(batch_size, 4)
goal = np.random.rand(batch_size, 4)
prior = Maze2DGoalPrior(6, horizon)

cond = {}
cond[0] = torch.tensor([[3,2,0,0]])
cond[horizon-1] = torch.tensor([[1,1,0,0]])

In [40]:
print(prior(cond)[:,:,4:])

[[[ 0.          0.        ]
  [ 0.51848097  0.12962024]
  [ 1.64991698  0.41247924]
  [ 3.68851722  0.92212931]
  [ 6.67015945  1.66753986]
  [10.09011346  2.52252837]
  [12.90393217  3.22598304]
  [14.00227008  3.50056752]
  [12.90393217  3.22598304]
  [10.09011346  2.52252837]
  [ 6.67015945  1.66753986]
  [ 3.68851722  0.92212931]
  [ 1.64991698  0.41247924]
  [ 0.51848097  0.12962024]
  [ 0.          0.        ]]]


In [14]:
start, goal = cond[0].detach().cpu().numpy(), cond[horizon-1].detach().cpu().numpy()

In [15]:
start

array([ 5, 15,  0,  0])

In [16]:
goal

array([1, 1, 0, 0])